In [1]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import tqdm
import Utility
import SQLSentence

%load_ext autoreload
%autoreload 1
%aimport SQLSentence
%aimport Utility

In [2]:
# Constants
TABLE_CODE = 'stock.stock_code'
TABLE_TYPE = 'stock.stock_type'
TABLE_CODE_TYPE = 'stock.stock_code_type'
OFFSET_TARGET = -1

target = 0.03 # Target rise percentage value

window_length_train = 20
window_length_target = 5

In [98]:
# Build Basic Data

connect = Utility.connect_to_database()
main_data = Utility.GetAllData(connect)
list_stock_type = (SQLSentence.QuerySQL(TABLE_CODE_TYPE, connect, ['distinct stock_type_id']))['stock_type_id'].tolist()
df_code_name = SQLSentence.QuerySQL(TABLE_CODE, connect)
dict_code_name = dict(zip(df_code_name.code, df_code_name.name))
dict_type_name = ((SQLSentence.QuerySQL(TABLE_TYPE, connect)).drop(['id'], axis=1)).to_dict()['name']

del df_code_name

In [99]:
class StockDataset(Dataset):
    def __init__(self, data, target, window_length_train, window_length_target):
        self.data = data
        self.target = target
        self.window_length_train = window_length_train
        self.window_length_target = window_length_target

        self.index_col_open = self.data.columns.get_loc("open")
        self.index_col_close = self.data.columns.get_loc("close")

        self.data = self.data.drop(['code', 'date'], axis=1).to_numpy()

        #處理資料、新增指標

    def __getitem__(self, index):
        data = torch.tensor(self.data[index:index+self.window_length_train], dtype=torch.float64).flatten()
        label = torch.tensor((self.data[index+self.window_length_train+self.window_length_target+OFFSET_TARGET][self.index_col_close] \
                              -self.data[index+self.window_length_train][self.index_col_open]) \
                                /self.data[index+self.window_length_train][self.index_col_open] >= self.target)
        return data, label
    
    def __len__(self):
        return len(self.data) - self.window_length_train - self.window_length_target

In [ ]:
for type_id in tqdm.tqdm(list_stock_type):
    list_unique_code_from_data = (SQLSentence.GetCodeByTypeId(type_id, connect))['code'].tolist()
    for code in list_unique_code_from_data:
        dataset = StockDataset(main_data[main_data['code'] == code], target, window_length_train, window_length_target)
        break